In [ ]:
import numpy as np
import scipy.optimize as sciop
import sympy
import matplotlib.pyplot as plt
from sympy.solvers import solve
from sympy.solvers.solveset import nonlinsolve
from sympy import Symbol
from sympy.core.symbol import symbols

In [ ]:
A=1
R0=6
av_d=1.5
av_b=2
β=0.383
ν=1
E=1000
k=30

In [ ]:
%matplotlib notebook
from matplotlib import colors

fig = plt.figure()
ax = plt.axes(projection='3d')

def C_I_minimal(ℓ,ζ):
#     return ζ**2*ℓ**2/2+2*β*ℓ*(A-ζ*ℓ)+4*R0*A/(E*ν*ℓ)
    return ζ**2*ℓ**2/2+2*β*ℓ*(A-ζ*ℓ)+2*R0/(k*np.pi)*(A**2-ζ**2*ℓ**2)

def C_I_minimum_0l():
    return 2*R0/(k*np.pi)*A**2

def C_I_minimal_heavy(ℓ,ζ):
    return ζ*ℓ*(ζ*ℓ-np.heaviside(ζ*ℓ-A,1)*(ζ*ℓ-1))/2+2*β*ℓ*(np.heaviside(A-ζ*ℓ,1)*(A-ζ*ℓ))+4*R0*A/(E*ν*ℓ)
#     return ζ*ℓ*(ζ*ℓ-np.heaviside(ζ*ℓ-A,1)*(ζ*ℓ-1))/2
#     return 2*β*ℓ*(np.heaviside(A-ζ*ℓ,1))*(A-ζ*ℓ)
#     return ζ*ℓ*(ζ*ℓ-np.heaviside(ζ*ℓ-A,1)*(ζ*ℓ-1))/2+2*β*ℓ*(np.heaviside(A-ζ*ℓ,1)*(A-ζ*ℓ))+2*R0/(k*np.pi)*(A**2-ζ**2*ℓ**2)

ℓs = np.linspace(0.01,1,50)
ζs = np.linspace(0,2,50)

X,Y = np.meshgrid(ℓs,ζs)

ζ_min = 2*β*av_d/(av_d-4*R0*av_b/(np.pi*k))
ℓ_min = A/ζ_min
# ℓ_min = av_b*(4*R0*ζ_min/(k*np.pi)-2*β*A)/(av_d*(ζ_min-4*β))
print(ζ_min,ℓ_min)

Z = C_I_minimal(X,Y)
mask = X*Y > A
Z[mask] = 0
# X[mask] = np.nan
# Y[mask] = np.nan
# Z[mask] = np.nan
# Z = np.ma.masked_where(X*Y<A, Z)

ax.plot_surface(X,Y,Z, cmap="viridis", norm=colors.LogNorm())
# ax.scatter(ℓ_min, ζ_min, C_I_minimal(ℓ_min,ζ_min), c='r', s=100)
# ax.plot([ℓs[0], ℓs[-1]], [ζ_min, ζ_min], [C_I_minimal(ℓ_min,ζ_min),C_I_minimal(ℓ_min,ζ_min)], c='r', ls='--')
# ax.plot([ℓ_min, ℓ_min], [ζs[0], ζs[-1]], [C_I_minimal(ℓ_min,ζ_min),C_I_minimal(ℓ_min,ζ_min)], c='r', ls='--')
# ax.plot([ℓ_min, ℓ_min], [ζ_min, ζ_min], [0,2], c='r', ls='--')
# ax.plot_surface(X,Y,np.full_like(X,C_I_minimum_0l()), cmap="viridis", norm=colors.LogNorm())
# ax.scatter(A/(2*β), 2*β, C_I_minimal(A/(2*β), 2*β), c='r')
ax.set_xlabel("ℓ")
ax.set_ylabel("ζ")

plt.show()

In [ ]:
min_idx_x = np.argmin(Z)//50
min_idx_y = np.argmin(Z)%50
print(min_idx_x, min_idx_y)
print(np.min(Z))
print(min_idx_x, min_idx_y)
print(X[min_idx_x, min_idx_y], Y[min_idx_x, min_idx_y])

In [ ]:
fig, ax = plt.subplots()

def C_I_minimal_projection(ℓ):
#     return ζ_min**2*ℓ**2/2+2*β*ℓ*(A-ζ_min*ℓ)+4*R0*A/(E*ν*ℓ)
    return ℓ-np.heaviside(ℓ-A,1)*(ℓ-1)
#     return np.heaviside(A-ℓ,1)*(A-ℓ)
#     return ℓ-np.heaviside(ℓ-A,1)*(ℓ-1) + np.heaviside(A-ℓ,1)*(A-ℓ)

ℓs = np.linspace(0.1,3,100)

# ax.scatter(ζ_min, ℓ_min, C_I_minimal(ℓ_min,ζ_min), c='r')
print(av_b*(4*R0*ζ_min/(k*np.pi)-2*β*A)/(av_d*(ζ_min-4*β)))
ax.plot(ℓs, C_I_minimal_projection(ℓs))
ax.set_xlabel("ℓ")

plt.show()

In [ ]:
def C_I_minimal(ℓ,ζ,μ):
    return ζ**2*ℓ**2/2+2*β*ℓ*(A-ζ*ℓ)+4*R0*μ*A/(E*ν*ℓ)

plt.figure()

ℓs = np.linspace(0.1,3,100)
plt.plot(ℓs, C_I_minimal(ℓs, 2, 1))

plt.show()

# Minimizing with $\mu$ as param

In [ ]:
def C_I(x):
    ℓ,ζ,μ,λ = x
    return 2/A**2*(av_d/av_b*(ζ**2*ℓ**2/2+2*β*ℓ*(A-ζ*ℓ))+4*R0*μ*A/(E*ν*ℓ))

def C_I_min(μ):
    return av_d/av_b*(1-1000000)/(A**2*E*ν*(av_d/av_b+4*R0/(np.pi*k)))

def system(x):
    ℓ,ζ,μ,λ = x
    eq1 = 2/A**2*(av_d/av_b*(ℓ*ζ**2+2*β*A-4*ζ*β*ℓ)-4*R0*μ*A/(ν*E*ℓ**2)) + λ*E/np.pi*(A-3*ζ**2*ℓ**2/A)
    eq2 = 4*R0/(A*E*ℓ)-λ*k
    eq3 = 2/A**2*(av_d/av_b*(ζ*ℓ**2-2*β*ℓ**2))-2*λ*ζ*ℓ**3*E/(np.pi*A)
    eq4 = E*ℓ/(np.pi*A)*(A**2-(ζ*ℓ)**2)-2*k*μ/ν
    return eq1, eq2, eq3, eq4

print(C_I((A*(4*R0*av_b + av_d*k*np.pi)/(2*av_d*k*np.pi*β), 2*av_d*k*np.pi*β/(4*R0*av_b + av_d*k*np.pi),1,1)))
print(C_I_min(1))

In [ ]:
def lambdaAnalytical(ℓ):
    return -4*R0/(k*A*E*ℓ)

def zetaAnalytical():
    return 2*β*av_d/(av_d+4/np.pi*av_b*R0/k)

def muAnalytical():
    return 0

def ellAnalytical():
    return A*(av_d+4/np.pi*av_b*R0/k)/(2*β*av_d)

print(ellAnalytical())
print(zetaAnalytical())
print(muAnalytical())
print(lambdaAnalytical(ellAnalytical()))
print(C_I([ellAnalytical(), zetaAnalytical(), muAnalytical(), lambdaAnalytical(ellAnalytical())]))

In [ ]:
sol = sciop.newton_krylov(system, [1.,1,1,1])
print(sol)
print(C_I(sol))

# Minimizing without $\mu$ as param

In [ ]:
μ=1

def system(x):
    ℓ,ζ,λ = x
    eq1 = 2/A**2*(av_d/av_b*(ℓ*ζ**2+2*β*A-4*ζ*β*ℓ)-4*R0*μ*A/(ν*E*ℓ**2)) + λ*E/np.pi*(A-3*ζ**2*ℓ**2/A)
    eq2 = 2/A**2*(av_d/av_b*(ζ*ℓ**2-2*β*ℓ**2))-2*λ*ζ*ℓ**3*E/(np.pi*A)
    eq3 = E*ℓ/(np.pi*A)*(A**2-(ζ*ℓ)**2)-2*k*μ/ν
    return eq1, eq2, eq3

sol = sciop.newton_krylov(system, [0.5,1,1])
print(sol)
print(sol[0]*sol[1])
pi=np.pi
λ=sol[2]
zeta_analytical = (A**3*E*av_b*k*λ**2*μ + 2*av_d*pi*β*ν - sqrt(A**3*E*av_b*λ*(A**3*E*av_b*k**2*λ**3*μ**2 + 4*av_d*k*pi*β*λ*μ*ν + av_d*pi*ν**2)))/(av_d*pi*ν)
print(zeta_analytical)
print(system(sol))
print(system([sol[0], zeta_analytical, sol[2]]))

# Solve eqs symbolically

In [ ]:
sympy.init_printing()

ℓ,ζ,λ = symbols("ℓ, ζ, λ", real=True)

pi = Symbol("pi", real=True)

A, av_d, av_b, β, R0, μ, ν, E, k, B = symbols("A, av_d, av_b, β, R0, μ, ν, E, k, B", real=True)

In [ ]:
eq1 = 2/A**2*(av_d/av_b*(ℓ*ζ**2+2*β*A-4*ζ*β*ℓ)-4*R0*μ*A/(ν*E*ℓ**2)) + λ*E/pi*(A-3*ζ**2*ℓ**2/A)
eq2 = 2/A**2*(av_d/av_b*(ζ*ℓ**2-2*β*ℓ**2))-2*λ*ζ*ℓ**3*E/(pi*A)
eq3 = E*ℓ/(pi*A)*(A**2-(ζ*ℓ)**2)-2*k*μ/ν

# eq5 = ℓ**3-A/(2*β)*ℓ**2+R0*μ*A*av_b/(k*pi*β**2)

# test = solve(eq5, ℓ)
# display(test)

display(eq1)
# sol = nonlinsolve([eq1,eq2,eq3], [ℓ,ζ,λ])

In [ ]:
eq3_pluggIn_zeta = E*ℓ/(pi*A)*(A**2-(2*β*av_d*ℓ/(av_d-av_b*A*λ*ℓ*E/pi))**2)-2*k*μ/ν
eq3_pluggIn_ell = av_d/(av_b*A**2*λ)*(ζ-2*β)*(A**2-(pi*av_d*(ζ-2*β)/(A*λ*E*av_b))**2)-2*k*μ*ζ/ν
eq_combining_zetas = 1/ℓ*sympy.sqrt(A**2-2*μ*k*pi*A/(E*ℓ*ν)) - 2*β*av_b/(av_d-av_b*A*λ*ℓ*E/pi)
eq_lambda_of_ell = -2*pi*β/(A*E*sympy.sqrt(A**2 - 2*A*k*pi*μ/(E*ν*ℓ))) + av_d*pi/(A*E*av_b*ℓ) - λ

display(eq_lambda_of_ell)
solution = sympy.solve(eq_lambda_of_ell,ℓ)
print(solution)
display(solution)

In [ ]:
display(solution)
print(type(solution))

## Without constraint

In [ ]:
eq1 = av_d/av_b*(ζ**2*ℓ+2*β*(A-2*ζ*ℓ))-4*R0*ζ**2*ℓ/(k*pi)
eq2 = av_d/av_b*ℓ**2*(ζ-2*β)-4*R0*ζ*ℓ**2/(k*pi)
C_I = 2/A**2*(av_d/av_b*(ζ**2*ℓ**2/2+2*β*ℓ*(A-ζ*ℓ))-4*R0*μ*A/(E*ν*ℓ))

sol = sympy.solve([eq1,eq2], [ℓ,ζ])

In [ ]:
display(sol)

In [ ]:
eq1 = av_d/av_b*(ζ**2*ℓ+2*β*(A-2*ζ*ℓ))-4*R0*μ*A/(E*ν*ℓ**2)
eq2 = av_d/av_b*ℓ**2*(ζ-2*β)

sol = sympy.solve([eq1,eq2], [ℓ,ζ])
display(sol)

In [ ]:
e, η = symbols("e, η", real=True)

eq1 = 2/A**2*(av_d/av_b*(ℓ*ζ**2+2*β*A-4*ζ*β*ℓ)-4*R0*μ*A/(ν*E*ℓ**2)) + λ*E/pi*(A-3*ζ**2*ℓ**2/A)
eq2 = 8*R0/(A*E*ℓ) - 2*λ*k + η
eq3 = 2/A**2*(av_d/av_b*(ζ*ℓ**2-2*β*ℓ**2))-2*λ*ζ*ℓ**3*E/(pi*A)
eq4 = E*ℓ/(pi*A)*(A**2-(ζ*ℓ)**2)-2*k*μ/ν
eq5 = μ/ν-e

In [ ]:
sol = sympy.solve([eq1,eq2,eq3,eq4,eq5], [ℓ,ζ,μ,λ,η])
display(sol)

## Without performance crit.

In [ ]:
eq_for_ell = ℓ**3 - A/(2*β)*ℓ**2 + av_b*R0*μ*A/(av_d*β**2*E*ν)

a = A / (2 * β)
b = av_b * R0 * μ * A / (av_d * β**2 * E * ν)
# a,b = symbols("a, b", real=True)

p = -a**2/3
q = b - 2*a**3/27

# p,q = symbols("p, q", real=True)

In [ ]:
# bla = (ℓ+a/3)**3-a*(ℓ+a/3)**2+b
# display(sympy.simplify(bla))

In [ ]:
display(eq_for_ell)
sol = sympy.solve(eq_for_ell, ℓ)
sol2 = sympy.solve(ℓ ** 3 - a * ℓ ** 2 + b, ℓ)
sol3 = sympy.solve(ℓ ** 3 + p * ℓ + q, ℓ)
# display(analytic_sol)
# display(sol)

discriminant = (q / 2) ** 2 + (p / 3) ** 3
display(
    "Value of discriminant: ",
    discriminant.subs(
        [
            (ν, 1),
            (E, 500000),
            (R0, 800 / 130),
            (μ, 10),
            (ζ, 1),
            (β, 1 / 6 * (np.sqrt(2) + np.log(1 + np.sqrt(2)))),
            (av_b, 2),
            (av_d, 1.3744),
            (A, 1 / 4),
        ]
    ),
)

analytic_sol = (
    sympy.cbrt(-q / 2 - sympy.sqrt((q / 2) ** 2 + (p / 3) ** 3))
    + sympy.cbrt(-q / 2 + sympy.sqrt((q / 2) ** 2 + (p / 3) ** 3))
    + a / 3
)
display(sympy.simplify(analytic_sol/(A/2)))

In [ ]:
# display(sol)

In [ ]:
# display(analytic_sol)

In [ ]:
# display(
#     "Solution: ",
#     analytic_sol.subs(
#         [
#             (ν, 1),
#             (E, 500000),
#             (R0, 800 / 130),
#             (μ, 10),
#             (ζ, 1),
#             (β, 1 / 6 * (np.sqrt(2) + np.log(1 + np.sqrt(2)))),
#             (av_b, 2),
#             (av_d, 2),
#             (A, 1 / 4),
#         ]
#     ),
# )
print("Approximate Solutions: ")
for s in sol:
    display(
#         analytic_sol.evalf(
        s.evalf(
            subs={
                ν: 1,
                E: 500000,
                R0: 800 / 130,
                μ: 10,
                ζ: 1,
                β: 1 / 6 * (np.sqrt(2) + np.log(1 + np.sqrt(2))),
                av_b: 2,
                av_d: 1.3744,
                A: 1 / 4,
            }
        )/(1/8),
    )

In [ ]:
display(
    "Approximate Solution: ",
#     analytic_sol.evalf(
    (analytic_sol).evalf(
        subs={
            ν: 1,
            E: 500000,
            R0: 800 / 130,
            μ: 10,
            ζ: 1,
            β: 1 / 6 * (np.sqrt(2) + np.log(1 + np.sqrt(2))),
            av_b: 2,
            av_d: 2,
            A: 1 / 4,
            B: 300,
        }
    ),
)